# Boilerplate

In [68]:
# files
TRAINING_DIRECTORY = 'cnn/stories/'
EXTENSION = '.story'
MAX_FILES = 50

# tokenization
FILTERS = '!"#$%&()*+,-./:;=?@[\\]^_`{|}~\t\n'  # default minus >, <
OOV_TOKEN = '<unk>'
BEGIN_TOKEN = 1
END_TOKEN = 2
NUM_WORDS = 600

# MODEL_PARAMS
MAX_SEQUENCE_LEN = 100

# Read in files

In [69]:
import glob

In [70]:
FILES = glob.glob('%s/*%s' % (TRAINING_DIRECTORY, EXTENSION))
print(len(FILES))
FILES

92579


['cnn/stories/0001d1afc246a7964130f43ae940af6bc6c57f01.story',
 'cnn/stories/0002095e55fcbd3a2f366d9bf92a95433dc305ef.story',
 'cnn/stories/00027e965c8264c35cc1bc55556db388da82b07f.story',
 'cnn/stories/0002c17436637c4fe1837c935c04de47adb18e9a.story',
 'cnn/stories/0003ad6ef0c37534f80b55b4235108024b407f0b.story',
 'cnn/stories/0004306354494f090ee2d7bc5ddbf80b63e80de6.story',
 'cnn/stories/0005d61497d21ff37a17751829bd7e3b6e4a7c5c.story',
 'cnn/stories/0006021f772fad0aa78a977ce4a31b3faa6e6fe5.story',
 'cnn/stories/00083697263e215e5e7eda753070f08aa374dd45.story',
 'cnn/stories/000940f2bb357ac04a236a232156d8b9b18d1667.story',
 'cnn/stories/0009ebb1967511741629926ef9f5faea2bb6be24.story',
 'cnn/stories/000c835555db62e319854d9f8912061cdca1893e.story',
 'cnn/stories/000ca3fc9d877f8d4bb2ebd1d6858c69be571fd8.story',
 'cnn/stories/000cd1ee0098c4d510a03ddc97d11764448ebac2.story',
 'cnn/stories/000e009f6b1d954d827c9a550f3f24a5474ee82b.story',
 'cnn/stories/001097a19e2c96de11276b3cce11566ccfed0030.

In [71]:
FILES = FILES[:MAX_FILES]

# Define method for generating text from files

In [72]:
def preprocessor(text):
    # replace all occurences of multiple newlines and replace them
    # with a single newline padded with spaces so it is treated as a
    # token
    text = ' \n '.join(t for t in text.split('\n') if t)
    table = {ord(c): None for c in '<>'}
    text = text.translate(table)
    return text

In [73]:
def text_generator(files, preprocessor=None):
    for f in files:
        text = open(f).read()
        if preprocessor is not None:
            text = preprocessor(text)
        # remove highlights
        body, highlight1, *_ = text.split('@highlight')
        yield body, highlight1

In [74]:
def tokenize(text, tokenizer, maxlen, begin_token, end_token):
    tokens = tokenizer([text])[0]
    tokens = tokens[:maxlen-2]
    return [[begin_token] + tokens + [end_token]]

In [75]:
next(text_generator(FILES, preprocessor=preprocessor))

('It\'s official: U.S. President Barack Obama wants lawmakers to weigh in on whether to use military force in Syria. \n Obama sent a letter to the heads of the House and Senate on Saturday night, hours after announcing that he believes military action against Syrian targets is the right step to take over the alleged use of chemical weapons. \n The proposed legislation from Obama asks Congress to approve the use of military force "to deter, disrupt, prevent and degrade the potential for future uses of chemical weapons or other weapons of mass destruction." \n It\'s a step that is set to turn an international crisis into a fierce domestic political battle. \n There are key questions looming over the debate: What did U.N. weapons inspectors find in Syria? What happens if Congress votes no? And how will the Syrian government react? \n In a televised address from the White House Rose Garden earlier Saturday, the president said he would take his case to Congress, not because he has to -- but

# Initialize tokenizer

In [76]:
from keras.preprocessing.text import text_to_word_sequence, Tokenizer as _Tokenizer

class Tokenizer(_Tokenizer):
    def fit_on_texts(self, texts):
        """Updates internal vocabulary based on a list of texts.
        In the case where texts contains lists, we assume each entry of the lists
        to be a token.
        Required before using `texts_to_sequences` or `texts_to_matrix`.
        # Arguments
            texts: can be a list of strings,
                a generator of strings (for memory-efficiency),
                or a list of list of strings.
        """
        for text in texts:
            self.document_count += 1
            if self.char_level or isinstance(text, list):
                seq = text
            else:
                seq = text_to_word_sequence(text,
                                            self.filters,
                                            self.lower,
                                            self.split)
            for w in seq:
                if w in self.word_counts:
                    self.word_counts[w] += 1
                else:
                    self.word_counts[w] = 1
            for w in set(seq):
                if w in self.word_docs:
                    self.word_docs[w] += 1
                else:
                    self.word_docs[w] = 1

        wcounts = list(self.word_counts.items())
        wcounts.sort(key=lambda x: x[1], reverse=True)
        sorted_voc = [wc[0] for wc in wcounts]
        # note that index 0, 1, 2 is reserved, never assigned to an existing word
        self.word_index = dict(list(zip(sorted_voc, list(range(4, len(sorted_voc) + 4)))))
        self.word_index[self.oov_token] = 3

        for w, c in list(self.word_docs.items()):
            self.index_docs[self.word_index[w]] = c

    def texts_to_sequences_generator(self, texts):
        """Transforms each text in `texts` in a sequence of integers.
        Each item in texts can also be a list, in which case we assume each item of that list
        to be a token.
        Only top "num_words" most frequent words will be taken into account.
        Only words known by the tokenizer will be taken into account.
        # Arguments
            texts: A list of texts (strings).
        # Yields
            Yields individual sequences.
        """
        num_words = self.num_words
        for text in texts:
            if self.char_level or isinstance(text, list):
                seq = text
            else:
                seq = text_to_word_sequence(text,
                                            self.filters,
                                            self.lower,
                                            self.split)
            vect = []
            for w in seq:
                i = self.word_index.get(w)
                if i is not None and (self.num_words and i < self.num_words):
                    vect.append(i)
                elif self.oov_token is not None:
                    i = self.word_index.get(self.oov_token)
                    if i is not None:
                        vect.append(i)
            yield vect

In [77]:
TOKENIZER = Tokenizer(
    num_words=NUM_WORDS,
    filters=FILTERS,  # no newline
    oov_token=OOV_TOKEN)

In [78]:
gen = text_generator(FILES, preprocessor=preprocessor)

In [79]:
%%time
TOKENIZER.fit_on_texts(text for train_pair in gen for text in train_pair)

CPU times: user 47.6 ms, sys: 8.5 ms, total: 56.1 ms
Wall time: 60 ms


In [80]:
TOKENIZER.num_words

600

In [81]:
TOKENIZER.document_count

100

In [82]:
len(TOKENIZER.word_index), TOKENIZER.word_index

(6590,
 {'the': 4,
  'to': 5,
  'of': 6,
  'a': 7,
  'in': 8,
  'and': 9,
  'that': 10,
  'is': 11,
  'for': 12,
  'on': 13,
  'said': 14,
  'it': 15,
  'with': 16,
  'was': 17,
  'he': 18,
  'as': 19,
  'has': 20,
  'be': 21,
  'his': 22,
  'have': 23,
  'from': 24,
  'by': 25,
  'at': 26,
  'this': 27,
  'are': 28,
  'not': 29,
  'but': 30,
  'who': 31,
  'an': 32,
  'they': 33,
  'will': 34,
  'u': 35,
  'their': 36,
  'more': 37,
  'been': 38,
  'or': 39,
  'i': 40,
  's': 41,
  'we': 42,
  'year': 43,
  'had': 44,
  'were': 45,
  'she': 46,
  'people': 47,
  'than': 48,
  'cnn': 49,
  'also': 50,
  'one': 51,
  'about': 52,
  'all': 53,
  'which': 54,
  'her': 55,
  'united': 56,
  'new': 57,
  'after': 58,
  'states': 59,
  'government': 60,
  'if': 61,
  'state': 62,
  'other': 63,
  'can': 64,
  'what': 65,
  'isis': 66,
  'when': 67,
  'obama': 68,
  'there': 69,
  'would': 70,
  'no': 71,
  'its': 72,
  'so': 73,
  'into': 74,
  'court': 75,
  "it's": 76,
  'two': 77,
  'him'

In [83]:
index_to_word = {v: k for k, v in TOKENIZER.word_index.items()}
index_to_word[0] = '<pad>'
index_to_word[1] = '<begin>'
index_to_word[2] = '<end>'

In [84]:
sorted(index_to_word.items(), key=lambda x: x[0])

[(0, '<pad>'),
 (1, '<begin>'),
 (2, '<end>'),
 (3, '<unk>'),
 (4, 'the'),
 (5, 'to'),
 (6, 'of'),
 (7, 'a'),
 (8, 'in'),
 (9, 'and'),
 (10, 'that'),
 (11, 'is'),
 (12, 'for'),
 (13, 'on'),
 (14, 'said'),
 (15, 'it'),
 (16, 'with'),
 (17, 'was'),
 (18, 'he'),
 (19, 'as'),
 (20, 'has'),
 (21, 'be'),
 (22, 'his'),
 (23, 'have'),
 (24, 'from'),
 (25, 'by'),
 (26, 'at'),
 (27, 'this'),
 (28, 'are'),
 (29, 'not'),
 (30, 'but'),
 (31, 'who'),
 (32, 'an'),
 (33, 'they'),
 (34, 'will'),
 (35, 'u'),
 (36, 'their'),
 (37, 'more'),
 (38, 'been'),
 (39, 'or'),
 (40, 'i'),
 (41, 's'),
 (42, 'we'),
 (43, 'year'),
 (44, 'had'),
 (45, 'were'),
 (46, 'she'),
 (47, 'people'),
 (48, 'than'),
 (49, 'cnn'),
 (50, 'also'),
 (51, 'one'),
 (52, 'about'),
 (53, 'all'),
 (54, 'which'),
 (55, 'her'),
 (56, 'united'),
 (57, 'new'),
 (58, 'after'),
 (59, 'states'),
 (60, 'government'),
 (61, 'if'),
 (62, 'state'),
 (63, 'other'),
 (64, 'can'),
 (65, 'what'),
 (66, 'isis'),
 (67, 'when'),
 (68, 'obama'),
 (69, 'the

In [85]:
TOKENIZER.num_words = min(len(TOKENIZER.word_index)+1, TOKENIZER.num_words)

In [86]:
gen = text_generator(FILES)
x, y = next(gen)

In [87]:
len(x)

9442

In [88]:
x

'It\'s official: U.S. President Barack Obama wants lawmakers to weigh in on whether to use military force in Syria.\n\nObama sent a letter to the heads of the House and Senate on Saturday night, hours after announcing that he believes military action against Syrian targets is the right step to take over the alleged use of chemical weapons.\n\nThe proposed legislation from Obama asks Congress to approve the use of military force "to deter, disrupt, prevent and degrade the potential for future uses of chemical weapons or other weapons of mass destruction."\n\nIt\'s a step that is set to turn an international crisis into a fierce domestic political battle.\n\nThere are key questions looming over the debate: What did U.N. weapons inspectors find in Syria? What happens if Congress votes no? And how will the Syrian government react?\n\nIn a televised address from the White House Rose Garden earlier Saturday, the president said he would take his case to Congress, not because he has to -- but 

In [89]:
x_sequence = tokenize(
    x,
    TOKENIZER.texts_to_sequences,
    maxlen=MAX_SEQUENCE_LEN,
    begin_token=BEGIN_TOKEN,
    end_token=END_TOKEN)
x_sequence

[[1,
  76,
  144,
  35,
  41,
  96,
  581,
  68,
  457,
  3,
  5,
  3,
  8,
  13,
  150,
  5,
  263,
  95,
  222,
  8,
  80,
  68,
  3,
  7,
  3,
  5,
  4,
  3,
  6,
  4,
  159,
  9,
  264,
  13,
  192,
  404,
  582,
  58,
  3,
  10,
  18,
  515,
  95,
  165,
  83,
  88,
  3,
  11,
  4,
  171,
  516,
  5,
  114,
  87,
  4,
  3,
  263,
  6,
  405,
  177,
  4,
  3,
  3,
  24,
  68,
  3,
  303,
  5,
  3,
  4,
  263,
  6,
  95,
  222,
  5,
  3,
  3,
  304,
  9,
  3,
  4,
  3,
  12,
  237,
  3,
  6,
  405,
  177,
  39,
  63,
  177,
  6,
  3,
  3,
  76,
  7,
  516,
  10,
  11,
  2]]

In [90]:
y_sequence = tokenize(
    y,
    TOKENIZER.texts_to_sequences,
    maxlen=MAX_SEQUENCE_LEN,
    begin_token=BEGIN_TOKEN,
    end_token=END_TOKEN)
y_sequence

[[1, 88, 144, 68, 3, 5, 4, 207, 6, 4, 3, 272, 195, 160, 5, 121, 139, 2]]

In [91]:
[[index_to_word[i] for i in L] for L in x_sequence]

[['<begin>',
  "it's",
  'official',
  'u',
  's',
  'president',
  'barack',
  'obama',
  'wants',
  '<unk>',
  'to',
  '<unk>',
  'in',
  'on',
  'whether',
  'to',
  'use',
  'military',
  'force',
  'in',
  'syria',
  'obama',
  '<unk>',
  'a',
  '<unk>',
  'to',
  'the',
  '<unk>',
  'of',
  'the',
  'house',
  'and',
  'senate',
  'on',
  'saturday',
  'night',
  'hours',
  'after',
  '<unk>',
  'that',
  'he',
  'believes',
  'military',
  'action',
  'against',
  'syrian',
  '<unk>',
  'is',
  'the',
  'right',
  'step',
  'to',
  'take',
  'over',
  'the',
  '<unk>',
  'use',
  'of',
  'chemical',
  'weapons',
  'the',
  '<unk>',
  '<unk>',
  'from',
  'obama',
  '<unk>',
  'congress',
  'to',
  '<unk>',
  'the',
  'use',
  'of',
  'military',
  'force',
  'to',
  '<unk>',
  '<unk>',
  'prevent',
  'and',
  '<unk>',
  'the',
  '<unk>',
  'for',
  'future',
  '<unk>',
  'of',
  'chemical',
  'weapons',
  'or',
  'other',
  'weapons',
  'of',
  '<unk>',
  '<unk>',
  "it's",
  'a

In [92]:
[[index_to_word[i] for i in L] for L in y_sequence]

[['<begin>',
  'syrian',
  'official',
  'obama',
  '<unk>',
  'to',
  'the',
  'top',
  'of',
  'the',
  '<unk>',
  "doesn't",
  'know',
  'how',
  'to',
  'get',
  'down',
  '<end>']]

In [93]:
len(x_sequence), len(x_sequence[0])

(1, 100)

In [94]:
from keras.preprocessing.sequence import pad_sequences
pad_sequences(x_sequence)

array([[  1,  76, 144,  35,  41,  96, 581,  68, 457,   3,   5,   3,   8,
         13, 150,   5, 263,  95, 222,   8,  80,  68,   3,   7,   3,   5,
          4,   3,   6,   4, 159,   9, 264,  13, 192, 404, 582,  58,   3,
         10,  18, 515,  95, 165,  83,  88,   3,  11,   4, 171, 516,   5,
        114,  87,   4,   3, 263,   6, 405, 177,   4,   3,   3,  24,  68,
          3, 303,   5,   3,   4, 263,   6,  95, 222,   5,   3,   3, 304,
          9,   3,   4,   3,  12, 237,   3,   6, 405, 177,  39,  63, 177,
          6,   3,   3,  76,   7, 516,  10,  11,   2]], dtype=int32)

In [95]:
[[index_to_word[i] for i in x] for x in pad_sequences(x_sequence)]

[['<begin>',
  "it's",
  'official',
  'u',
  's',
  'president',
  'barack',
  'obama',
  'wants',
  '<unk>',
  'to',
  '<unk>',
  'in',
  'on',
  'whether',
  'to',
  'use',
  'military',
  'force',
  'in',
  'syria',
  'obama',
  '<unk>',
  'a',
  '<unk>',
  'to',
  'the',
  '<unk>',
  'of',
  'the',
  'house',
  'and',
  'senate',
  'on',
  'saturday',
  'night',
  'hours',
  'after',
  '<unk>',
  'that',
  'he',
  'believes',
  'military',
  'action',
  'against',
  'syrian',
  '<unk>',
  'is',
  'the',
  'right',
  'step',
  'to',
  'take',
  'over',
  'the',
  '<unk>',
  'use',
  'of',
  'chemical',
  'weapons',
  'the',
  '<unk>',
  '<unk>',
  'from',
  'obama',
  '<unk>',
  'congress',
  'to',
  '<unk>',
  'the',
  'use',
  'of',
  'military',
  'force',
  'to',
  '<unk>',
  '<unk>',
  'prevent',
  'and',
  '<unk>',
  'the',
  '<unk>',
  'for',
  'future',
  '<unk>',
  'of',
  'chemical',
  'weapons',
  'or',
  'other',
  'weapons',
  'of',
  '<unk>',
  '<unk>',
  "it's",
  'a

In [96]:
s = x_sequence[0]
s

[1,
 76,
 144,
 35,
 41,
 96,
 581,
 68,
 457,
 3,
 5,
 3,
 8,
 13,
 150,
 5,
 263,
 95,
 222,
 8,
 80,
 68,
 3,
 7,
 3,
 5,
 4,
 3,
 6,
 4,
 159,
 9,
 264,
 13,
 192,
 404,
 582,
 58,
 3,
 10,
 18,
 515,
 95,
 165,
 83,
 88,
 3,
 11,
 4,
 171,
 516,
 5,
 114,
 87,
 4,
 3,
 263,
 6,
 405,
 177,
 4,
 3,
 3,
 24,
 68,
 3,
 303,
 5,
 3,
 4,
 263,
 6,
 95,
 222,
 5,
 3,
 3,
 304,
 9,
 3,
 4,
 3,
 12,
 237,
 3,
 6,
 405,
 177,
 39,
 63,
 177,
 6,
 3,
 3,
 76,
 7,
 516,
 10,
 11,
 2]

In [97]:
one_hot = TOKENIZER.sequences_to_matrix([[i] for i in s])

In [98]:
one_hot

array([[0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.]])

In [99]:
# only one per row
import numpy as np
np.argwhere(one_hot == 1)[:50]

array([[  0,   1],
       [  1,  76],
       [  2, 144],
       [  3,  35],
       [  4,  41],
       [  5,  96],
       [  6, 581],
       [  7,  68],
       [  8, 457],
       [  9,   3],
       [ 10,   5],
       [ 11,   3],
       [ 12,   8],
       [ 13,  13],
       [ 14, 150],
       [ 15,   5],
       [ 16, 263],
       [ 17,  95],
       [ 18, 222],
       [ 19,   8],
       [ 20,  80],
       [ 21,  68],
       [ 22,   3],
       [ 23,   7],
       [ 24,   3],
       [ 25,   5],
       [ 26,   4],
       [ 27,   3],
       [ 28,   6],
       [ 29,   4],
       [ 30, 159],
       [ 31,   9],
       [ 32, 264],
       [ 33,  13],
       [ 34, 192],
       [ 35, 404],
       [ 36, 582],
       [ 37,  58],
       [ 38,   3],
       [ 39,  10],
       [ 40,  18],
       [ 41, 515],
       [ 42,  95],
       [ 43, 165],
       [ 44,  83],
       [ 45,  88],
       [ 46,   3],
       [ 47,  11],
       [ 48,   4],
       [ 49, 171]])

# Define batch generator

In [100]:
def sequencer(tokens):
    return [tokens[:i] for i in range(1, len(tokens)+1)]

In [101]:
list(sequencer('a quick brown fox'))

['a',
 'a ',
 'a q',
 'a qu',
 'a qui',
 'a quic',
 'a quick',
 'a quick ',
 'a quick b',
 'a quick br',
 'a quick bro',
 'a quick brow',
 'a quick brown',
 'a quick brown ',
 'a quick brown f',
 'a quick brown fo',
 'a quick brown fox']

In [128]:
import random
import numpy as np

class BatchGenerator:
    def __init__(self, files, tokenizer, maxlen, batch_size, begin_token, end_token, epoch_end=None):
        self.files = files
        self.tokenizer = tokenizer
        self.maxlen = maxlen
        self.batch_size = batch_size
        self.begin_token = begin_token
        self.end_token = end_token
        self.epoch_end = epoch_end
        
    def generate(self):
        steps = []
        while True:
            random.shuffle(self.files)
            for encode_text, decode_text in self.iter_files(self.files):
                encode_tokens = self.tokenize(encode_text)
                decode_tokens = self.tokenize(decode_text)

                for seq_tokens in self.sequence(decode_tokens):
                    steps.append((encode_tokens, seq_tokens))
                    
                while len(steps) >= self.batch_size:
                    batch = steps[:self.batch_size]
                    encoder_input, decoder_input = zip(*batch)
                    x1 = pad_sequences(encoder_input, maxlen=self.maxlen)
                    x2 = pad_sequences(decoder_input, maxlen=self.maxlen)
                    y = self.tokenizer.sequences_to_matrix([[i] for seq in x2 for i in seq])
                    y = y.reshape((self.batch_size, self.maxlen, self.tokenizer.num_words))
                    
                    # offset
                    X = [x1[:-1], x2[:-1]]
                    y = y[1:]
                    yield X, y
                    
                    # reset
                    steps = steps[self.batch_size:]
            yield self.epoch_end
        
    def preprocess(self, text):
        # replace all occurences of multiple newlines and replace them
        # with a single newline padded with spaces so it is treated as a
        # token
        text = ' \n '.join(t for t in text.split('\n') if t)
        table = {ord(c): None for c in '<>'}
        text = text.translate(table)
        return text

    def iter_files(self, files):
        for f in files:
            text = open(f).read()
            text = self.preprocess(text)
            # remove highlights
            body, highlight1, *_ = text.split('@highlight')
            yield body, highlight1
        
    def tokenize(self, text):
        tokens = self.tokenizer.texts_to_sequences([text])[0]
        tokens = tokens[:self.maxlen-2]
        return [self.begin_token] + tokens + [self.end_token]

    def sequence(self, tokens):
        return [tokens[:i] for i in range(1, len(tokens)+1)] 

In [129]:
batch_gen = BatchGenerator(
    files=FILES,
    tokenizer=TOKENIZER,
    maxlen=MAX_SEQUENCE_LEN,
    batch_size=32,
    begin_token=BEGIN_TOKEN,
    end_token=END_TOKEN).generate()

In [130]:
X, y = next(batch_gen)

In [131]:
X[0].shape, X[1].shape, y.shape

((31, 100), (31, 100), (31, 100, 600))

In [132]:
x1, x2 = X
x1

array([[  1,  49,   3, ...,   8,   3,   2],
       [  1,  49,   3, ...,   8,   3,   2],
       [  1,  49,   3, ...,   8,   3,   2],
       ...,
       [  1,  57, 396, ..., 489, 523,   2],
       [  1,  57, 396, ..., 489, 523,   2],
       [  1,  57, 396, ..., 489, 523,   2]], dtype=int32)

In [133]:
y

array([[[1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.],
        ...,
        [1., 0., 0., ..., 0., 0., 0.],
        [0., 1., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 1., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       ...,

       [[1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0.

In [134]:
[[index_to_word.get(i, '<pad>') for i in x] for x in x1]

[['<begin>',
  'cnn',
  '<unk>',
  '<unk>',
  'at',
  'a',
  'syrian',
  'government',
  '<unk>',
  'national',
  '<unk>',
  'meeting',
  'sunday',
  'criticized',
  'recent',
  '<unk>',
  'by',
  'the',
  "country's",
  'security',
  'forces',
  '<unk>',
  'for',
  'an',
  'end',
  'to',
  'violence',
  'against',
  'protesters',
  "syria's",
  '<unk>',
  'president',
  '<unk>',
  'the',
  '<unk>',
  'university',
  'meeting',
  'between',
  'officials',
  'and',
  'members',
  'of',
  'the',
  'opposition',
  'as',
  'a',
  'step',
  'toward',
  '<unk>',
  'a',
  'democratic',
  'nation',
  'we',
  '<unk>',
  'that',
  'at',
  'the',
  'end',
  'of',
  'this',
  '<unk>',
  'meeting',
  'to',
  '<unk>',
  'the',
  '<unk>',
  'of',
  'syria',
  'to',
  'a',
  '<unk>',
  'democratic',
  'nation',
  'where',
  'all',
  'citizens',
  'are',
  '<unk>',
  'by',
  '<unk>',
  'and',
  '<unk>',
  'in',
  'the',
  '<unk>',
  'of',
  'the',
  'future',
  'of',
  'their',
  'country',
  '<unk>',


In [ ]:
# only one per row
import numpy as np
ys = np.argwhere(y[0] == 1)

In [158]:
import numpy as np
for j in range(0, len(y), 5):
    ys = np.argwhere(y[j] == 1)
    assert len(ys) == len({row for row, idx in ys})
    print(' '.join(index_to_word[idx] for row, idx in ys))

<pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <begin> new
<pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad

# Training

In [177]:
N_HEADS = 8
N_LAYERS = 6
D_MODEL = 64*N_HEADS
VOCAB_SIZE = TOKENIZER.num_words
WARMUP_STEPS = 200

In [163]:
batch_gen = BatchGenerator(
    files=FILES,
    tokenizer=TOKENIZER,
    maxlen=MAX_SEQUENCE_LEN,
    batch_size=32,
    begin_token=BEGIN_TOKEN,
    end_token=END_TOKEN).generate()

In [172]:
# loop over batch generator until we hit the end of the epoch
# to calculate number of batches in epoch and compute some
# stats along the way
steps_per_epoch = 0
for batch in batch_gen:
    if batch is None:
        break
    steps_per_epoch += 1

In [173]:
print('steps per epoch', steps_per_epoch)

steps per epoch 23


In [174]:
train_gen = (X for X in batch_gen if not X == 0)

In [175]:
from keras.callbacks import TerminateOnNaN
callbacks = [TerminateOnNaN()]

In [178]:
from model import Transformer
model = Transformer(
        n_heads=N_HEADS, encoder_layers=N_LAYERS, decoder_layers=N_LAYERS,
        d_model=D_MODEL, vocab_size=VOCAB_SIZE, sequence_len=MAX_SEQUENCE_LEN,
        layer_normalization=True, dropout=True,
        residual_connections=True)

In [179]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
decoder_input (InputLayer)      (None, None)         0                                            
__________________________________________________________________________________________________
encoder_input (InputLayer)      (None, None)         0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, 100, 512)     307200      encoder_input[0][0]              
                                                                 decoder_input[0][0]              
__________________________________________________________________________________________________
positional_encoding_1 (Position (None, 100, 512)     0           embedding[0][0]                  
          

In [398]:
# import keras.backend as K
# def loss(y_true, y_pred):
#    return K.categorical_crossentropy(y_true[:,-1:,:], y_pred[:,-1:,:])

In [399]:
loss = 'categorical_crossentropy'

In [400]:
class LRScheduler:
    def __init__(self, d_model, warmup_steps):
        self.d_model = d_model
        self.warmup_steps = warmup_steps
        self.epoch = 1

    def lr(self, epoch):
        lr = self.d_model**-.5 * min(self.epoch**-.5, epoch*(self.warmup_steps**-1.5))
        self.epoch += 1
        return lr
lr_scheduler = LRScheduler(d_model, warmup_steps)

In [401]:
from keras.callbacks import LearningRateScheduler
# callbacks.append(LearningRateScheduler(lr_scheduler.lr))

In [402]:
from keras.optimizers import adam
model.compile(loss=loss, optimizer=adam(lr=1e-4))

In [403]:
# from keras import backend as K
# old_lr = K.get_value(model.optimizer.lr)
# K.set_value(model.optimizer.lr, 1e-4)

In [404]:
n_epochs = 1000
model.fit_generator(
    train_gen, steps_per_epoch=steps_per_epoch,
    epochs=n_epochs, callbacks=callbacks)

Epoch 1/1000
83/83 [==============================] - 31s 373ms/step - loss: 6.1677
Epoch 2/1000
83/83 [==============================] - 13s 155ms/step - loss: 5.8673
Epoch 3/1000
83/83 [==============================] - 13s 156ms/step - loss: 5.6662
Epoch 4/1000
83/83 [==============================] - 13s 155ms/step - loss: 5.5196
Epoch 5/1000
83/83 [==============================] - 13s 157ms/step - loss: 5.4028
Epoch 6/1000
83/83 [==============================] - 13s 158ms/step - loss: 5.3272
Epoch 7/1000
83/83 [==============================] - 13s 156ms/step - loss: 5.2618
Epoch 8/1000
83/83 [==============================] - 13s 156ms/step - loss: 5.1847
Epoch 9/1000
83/83 [==============================] - 13s 156ms/step - loss: 5.1433
Epoch 10/1000
83/83 [==============================] - 13s 156ms/step - loss: 5.0959
Epoch 11/1000
83/83 [==============================] - 13s 156ms/step - loss: 5.0487
Epoch 12/1000
83/83 [==============================] - 13s 154ms/step - lo

83/83 [==============================] - 13s 157ms/step - loss: 4.2119
Epoch 193/1000
83/83 [==============================] - 13s 157ms/step - loss: 4.2916
Epoch 194/1000
83/83 [==============================] - 13s 155ms/step - loss: 4.2718
Epoch 195/1000
83/83 [==============================] - 13s 156ms/step - loss: 4.2281
Epoch 196/1000
83/83 [==============================] - 13s 155ms/step - loss: 4.0842
Epoch 197/1000
83/83 [==============================] - 13s 156ms/step - loss: 4.4146
Epoch 198/1000
83/83 [==============================] - 13s 158ms/step - loss: 4.1248
Epoch 199/1000
83/83 [==============================] - 13s 156ms/step - loss: 4.2627
Epoch 200/1000
83/83 [==============================] - 13s 159ms/step - loss: 4.4229
Epoch 201/1000
83/83 [==============================] - 13s 155ms/step - loss: 4.2359
Epoch 202/1000
83/83 [==============================] - 13s 157ms/step - loss: 4.2389
Epoch 203/1000
83/83 [==============================] - 13s 157ms/ste

83/83 [==============================] - 13s 156ms/step - loss: 4.3498
Epoch 383/1000
83/83 [==============================] - 13s 155ms/step - loss: 4.1534
Epoch 384/1000
83/83 [==============================] - 13s 156ms/step - loss: 4.3944
Epoch 385/1000
83/83 [==============================] - 13s 156ms/step - loss: 4.2010
Epoch 386/1000
83/83 [==============================] - 13s 157ms/step - loss: 4.2862
Epoch 387/1000
83/83 [==============================] - 13s 157ms/step - loss: 4.2597
Epoch 388/1000
83/83 [==============================] - 13s 156ms/step - loss: 4.2519
Epoch 389/1000
83/83 [==============================] - 13s 154ms/step - loss: 4.2110
Epoch 390/1000
83/83 [==============================] - 13s 157ms/step - loss: 4.2593
Epoch 391/1000
83/83 [==============================] - 13s 156ms/step - loss: 4.1078
Epoch 392/1000
83/83 [==============================] - 13s 157ms/step - loss: 4.3673
Epoch 393/1000
83/83 [==============================] - 13s 156ms/ste

83/83 [==============================] - 13s 158ms/step - loss: 4.3314
Epoch 573/1000
83/83 [==============================] - 13s 152ms/step - loss: 4.3353
Epoch 574/1000
83/83 [==============================] - 13s 155ms/step - loss: 4.0708
Epoch 575/1000
83/83 [==============================] - 13s 155ms/step - loss: 4.2803
Epoch 576/1000
83/83 [==============================] - 13s 158ms/step - loss: 4.2928
Epoch 577/1000
83/83 [==============================] - 13s 154ms/step - loss: 4.2818
Epoch 578/1000
83/83 [==============================] - 13s 154ms/step - loss: 4.0417
Epoch 579/1000
83/83 [==============================] - 13s 154ms/step - loss: 4.3572
Epoch 580/1000
83/83 [==============================] - 13s 157ms/step - loss: 4.1611
Epoch 581/1000
83/83 [==============================] - 13s 157ms/step - loss: 4.2172
Epoch 582/1000
83/83 [==============================] - 13s 155ms/step - loss: 4.3853
Epoch 583/1000
83/83 [==============================] - 13s 155ms/ste

83/83 [==============================] - 13s 157ms/step - loss: 4.3508
Epoch 763/1000
83/83 [==============================] - 13s 155ms/step - loss: 4.2670
Epoch 764/1000
83/83 [==============================] - 13s 156ms/step - loss: 4.3202
Epoch 765/1000
83/83 [==============================] - 13s 154ms/step - loss: 4.2152
Epoch 766/1000
83/83 [==============================] - 13s 155ms/step - loss: 4.0144
Epoch 767/1000
83/83 [==============================] - 13s 156ms/step - loss: 4.5115
Epoch 768/1000
83/83 [==============================] - 13s 156ms/step - loss: 3.9304
Epoch 769/1000
83/83 [==============================] - 13s 155ms/step - loss: 4.2439
Epoch 770/1000
83/83 [==============================] - 13s 155ms/step - loss: 4.4499
Epoch 771/1000
83/83 [==============================] - 13s 155ms/step - loss: 4.1880
Epoch 772/1000
83/83 [==============================] - 13s 154ms/step - loss: 4.2040
Epoch 773/1000
83/83 [==============================] - 13s 156ms/ste

83/83 [==============================] - 13s 157ms/step - loss: 3.9397
Epoch 953/1000
83/83 [==============================] - 13s 154ms/step - loss: 4.2589
Epoch 954/1000
83/83 [==============================] - 13s 156ms/step - loss: 4.4080
Epoch 955/1000
83/83 [==============================] - 13s 155ms/step - loss: 4.1978
Epoch 956/1000
83/83 [==============================] - 13s 154ms/step - loss: 4.2281
Epoch 957/1000
83/83 [==============================] - 13s 155ms/step - loss: 4.2783
Epoch 958/1000
83/83 [==============================] - 13s 153ms/step - loss: 4.1752
Epoch 959/1000
83/83 [==============================] - 13s 158ms/step - loss: 4.2613
Epoch 960/1000
83/83 [==============================] - 13s 154ms/step - loss: 4.0888
Epoch 961/1000
83/83 [==============================] - 13s 159ms/step - loss: 4.3022
Epoch 962/1000
83/83 [==============================] - 13s 155ms/step - loss: 4.1627
Epoch 963/1000
83/83 [==============================] - 13s 154ms/ste

In [78]:
seed = ['songbegin']
tokens = [[tokenizer.word_index[i] for i in seed]]
s = pad_sequences(tokens, maxlen=sequence_len)
for _ in range(30):
    X = [s, s]
    y_hat = model.predict(X)
    i = np.argmax(y_hat[0][-1])
    tokens[0].append(i)
    s = pad_sequences(tokens, maxlen=sequence_len)
' '.join([index_to_word.get(x, '<pad>') for x in tokens[0]])

'songbegin \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n'

In [ ]:
seed = ['you', 'are']
tokens = [[tokenizer.word_index[i] for i in seed]]
print(tokens)
s = pad_sequences(tokens, maxlen=sequence_len)
print(s)

In [ ]:
X, y = next(train_gen)

In [ ]:
x1, _ = X
[index_to_word.get(i, '<pad>') for i in x1[-1]]

In [ ]:
# X = [s, s]
y_hat = model.predict(X)
y_hat.shape

In [ ]:
[index_to_word.get(i, '<pad>') for i in np.argmax(y_hat[-1], axis=1)]

In [ ]:
np.argmax(y_hat[0], axis=1)